<a href="https://colab.research.google.com/github/june-oh/2023_AI_Academy_ASR/blob/main/6_nemo_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. NeMo fine-tuning

## NeMo (NVIDIA, 2019)
-  https://developer.nvidia.com/nvidia-nemo (homepage)
- https://github.com/NVIDIA/NeMo (source)
- open-source framework for developers to build and train state-of-the-art (SOTA) conversational AI models
- PyTorch, PyTorch Lightning을 기반으로 작성된 E2E toolkit
- SOTA model들의 pretrained model을 제공
- https://catalog.ngc.nvidia.com/ (ASR pretrained models)

## NeMo 설치

### pip를 이용한 설치
- pip install nemo_toolkit['all']

### Source code를 이용한 설치
- apt-get update && apt-get install -y libsndfile1 ffmpeg
- git clone https://github.com/NVIDIA/NeMo
- cd NeMo
- ./reinstall.sh

### NeMo Docker containers
- https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo
- docker pull nvcr.io/nvidia/nemo:22.11 (build container)
- docker run --runtime=nvidia -it --rm -v --shm-size=16g -p 8888:8888 -p 6006:6006 --ulimit memlock=-1 --ulimit stack=67108864 nvcr.io/nvidia/nemo:22.11 (docker run)

In [ ]:
# install NeMo

- `omegaconf` : yaml, json등의 configuration 파일을 읽고 쓸 수 있는 라이브러리, Nemo에서 기본적으로 사용
- `nemo.collections.asr` : NeMo ASR Class
- `nemo.utils.exp_manager` : 학습로그, conf 등에 사용되는 라이브러리

In [ ]:
from omegaconf import OmegaConf, open_dict

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
from nemo.utils import exp_manager

## pre-trained 모델 불러오기
- Quartznet model 사용
- https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/asr/models.html#quartznet
- SOTA성능을 보이는 모델은 아니지만, 파라미터 수가 작아 low resourced computing 환경에서 동작이 쉬움
- 500MB~1,000MB 사이의 SOTA 수준 모델에 비해 78MB의 적은 용량으로 동작 가능
- Librispeech test set 기준 4.19%의 word error rate (WER)
- https://arxiv.org/abs/1910.10261 (paper)


<img src = "https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/_images/quartz_vertical.png" height=700>

- https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_en_quartznet15x5 (pretrained model)


In [ ]:
# pretrained model load (model : "stt_en_quartznet15x5")

## Small data를 통한 모델 확인 (영어)

- `datasets.load_dataset` : 학습 및 테스트 데이터 관리 라이브러리 

In [ ]:
# ! pip install datasets

In [ ]:
# LibriSpeech dataset load
english_ds = load_dataset("kresnik/librispeech_asr_test", "clean")

In [ ]:
# check file list

In [ ]:
# check sample data

- `IPython.display` : IPython 위젯을 사용할 수 있는 라이브러리

In [ ]:
import IPython.display as ipd

In [ ]:
# listen audio file using ipd.Audio

In [ ]:
# transcribe using pretrained model

In [ ]:
# check result

In [ ]:
# compare with reference

- `jiwer` : CER, WER 등 음성인식 결과 평가 관련 라이브러리

In [ ]:
! pip install jiwer

In [ ]:
# calculate cer

## traning/test 데이터 불러오기 및 데이터 확인 (한국어)

- dataset : file, audio, text, speaker 정보

In [ ]:
# Zeroth-Korean dataset load
ds = load_dataset("kresnik/zeroth_korean", "clean")

In [ ]:
# check file list

In [ ]:
# check sample data

In [ ]:
# listen audio file using ipd.Audio

In [ ]:
# split train, validation, test set

NeMo의 ASR data preparation은 3가지의 정보 필요
- audio_filepath, duration, text

In [ ]:
# remove columns

In [ ]:
# rename column

In [ ]:
# get duration

- `force_ascii = False` : 한글이 깨지지 않도록 encoding
- `orient = "records` : {columns:value} 형태의 딕셔너리를 요소로 하는 리스트 형태

In [ ]:
# make json file

In [ ]:
# check json file

## 한국어 fine-tuning을 위한 모델 설정
- 영어로 학습된 모델을 소용량의 한국어를 이용해 원활히 tuning하기 위해  
model의 encoder 정보는 그대로 유지  
model의 deocder 정보(sequence of output unit representation)을 재학습하는 것이 효율적)
- 충분한 양의 데이터를 확보하지 못했거나, computing 환경이 부족할 때 사용하는 방법
- encoder 전체를 학습하지 않는 경우 normalization 문제가 발생할 수 있음  
Ex> 원본 모델이 학습한 음성 데이터와 새로운 학습 데이터의 볼륨(소리 크기) 차이가 많이 나는 문제  
이를 방지하기 위해 batch normalization 부분은 freeze하지 않음




In [ ]:
import torch
import torch.nn as nn

In [ ]:
# batch normalization unfreeze

In [ ]:
# encoder freeze

In [ ]:
# unfreeze apply

In [ ]:
# unfreeze check (requires_grad -> True : 가중치 학습)

## 한국어 ouput unit 설정 및 training 세팅
- 영어 알파벳으로 정의된 모델을 한국어 음절로 변경

In [ ]:
# check model config

In [ ]:
# extract characters

In [ ]:
# make vocab list

In [ ]:
# check vocab list

In [ ]:
# change model output unit

In [ ]:
import copy

In [ ]:
# copy configuration

In [ ]:
# setup train, validation configuration(manifest_filepath, labels, normalize_transcripts(대소문자), batch_size, num_workers, pin_memory, trim_silence)

In [ ]:
# setup data loader with new configs

In [ ]:
# print original optimizer + scheduler

In [ ]:
??torch.optim.Adam

- lr = 0.01
- betas = [0.95, 0.25]
- weight_decay = 0.001 (original weight decay)
- sched.warup_steps = None (remove default number of steps of warmup)
- sched.warup_ratio = 0.05 (5% warmup)
- sched.min_lr = 1e-5

In [ ]:
# setup optimizer

In [ ]:
# print original spec_augment

## 학습
- 학습 과정을 표시하는 metric 설정

In [ ]:
# use_cer & log_prediction

In [ ]:
import torch
import pytorch_lightning as ptl

- gpus, max_epochs, accumulate_grad_batches, checkpoint_callback, logger, log_every_n_steps, check_val_every_n_epoch

In [ ]:
# set trainer

In [ ]:
# setup model with the trainer

In [ ]:
# update the model's internal config

In [ ]:
import os

os.environ.pop('NEMO_EXPM_VERSION', None)

In [ ]:
# set log path

In [ ]:
# set exp_manager

In [ ]:
# set tensorboard

In [ ]:
# start train

In [ ]:
# save model

## 테스트

In [ ]:
# load trained model

In [ ]:
# transcribe using trained model

In [ ]:
# compare with reference

In [ ]:
# calculate cer

## NeMo에서 full recipe로 학습을 수행하는 방법
- 임의의 데이터에 대해 같은 모델을 처음부터 학습하는 방법
- Colab에서는 동작이 어려움
- 시간 소요가 큼
- https://github.com/NVIDIA/NeMo/tree/main/examples/asr/asr_ctc